<div class="details">
      <div class="resultTitle">
        <h3 itemprop="name" class="booktitle">
          <a itemprop="url" href="/works/OL17930368W?edition=ia%3Aatomichabitseasy0000clea" class="results">Atomic Habits</a>
        </h3>
      </div>
        <span itemprop="author" itemscope="" itemtype="https://schema.org/Organization" class="bookauthor"> by <a href="/authors/OL7422948A">James Clear</a> </span>
        <span class="resultStats">
          <span class="resultDetails">
            <span>
              First published in 2016
            </span>—
          </span>
        </span>
        Logged 41180 times All Time
    </div>

In [74]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [98]:
bookData = pd.DataFrame({
    'Title': [],
    'Author': [],
    'Year Published': [],
    'Number of Logs': []
})

def processPage(url):
    global bookData
    
    # Request the page and handle potential errors
    try:
        html = requests.get(url)
        html.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Failed to get website data: {e}")
        return

    # Find all book details
    soup = BeautifulSoup(html.text, 'html.parser')
    book_title = soup.find('a', class_='results').text

    # Extract author name
    author = soup.find('span', class_='bookauthor').find('a').text

    # Extract release year
    release_year = soup.find('span', class_='resultDetails').find('span').text.split()[-1]

    # Extract log count
    log_count = soup.find('div', class_='details').text.split("Logged")[-1].strip()


    bookData = pd.concat([bookData, pd.DataFrame({
        'Title': [book_title],
        'Author': [author],
        'Year Published': [release_year],
        'Number of Logs': [log_count]
    })], ignore_index=True)

for i in range(10):
    processPage(f'https://openlibrary.org/trending/forever?page={i+1}')

    
bookData  

,Title,Author,Year Published,Number of Logs
0,Atomic Habits,James Clear,2016,41181 times All Time
1,Ugly Love,Colleen Hoover,2014,6200 times All Time
2,"Red, White & Royal Blue",Casey McQuiston,2019,4186 times All Time
3,101 Essays That Will Change The Way You Think,Brianna Wiest,2018,3219 times All Time
4,A Little Life,Hanya Yanagihara,2015,2645 times All Time
5,Misery,Stephen King,1978,2256 times All Time
6,The Kite Runner,Khaled Hosseini,2003,1893 times All Time
7,Steve Jobs,Walter Isaacson,2011,1607 times All Time
8,Bible,Bible,1200,1455 times All Time
9,The BFG,Roald Dahl,1980,1376 times All Time


In [101]:
bookData = pd.DataFrame({
    'Title': [],
    'Author': [],
    'Year Published': [],
    'Number of Logs': [],
    'url': []
})

def processPage(url):
    global bookData
    
    # Request the page and handle potential errors
    try:
        html = requests.get(url)
        html.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Failed to get website data: {e}")
        return

    # Find all book details
    soup = BeautifulSoup(html.text, 'html.parser')
    bookDetails = soup.findAll('div', class_='details')

    for book in bookDetails:
        details = re.findall(r'[A-Z][^A-Z]*', book.get_text(strip=True))
        
        # Remove the last 2 elements
        del details[len(details)-2] 
        del details[len(details)-3] 

        # Clean up & process (some of) the data
        # ... remove 'by' in the books name and get the books name
        # ... and then strip out the integers to find when it was published and logged
        # ... and then remove the book name from the list
        
        i=0
        bookName = ""
        bookNameEndIndex = 0
        for i, detail in enumerate(details):
            if 'by' in detail: # Get rid of 'by' in the books name and log it
                details[i] = detail.replace('by', '').strip()
                bookName = ''.join(details[:i+1])
                bookNameEndIndex = i
            elif 'published' in detail or 'Logged' in detail:
                # Get all integers
                integers = re.findall(r'\d+', detail)
                if integers:
                    details[i] = integers[0] 
        del details[0:bookNameEndIndex+1]

        
        # Find the authors name
        # ... iterates through the remaining elements up until it hits an integer and stores the index in which it hit said integer
        # ... slice and join the list to get the authors name
        # ... handle 'and' cases
        # ... remove the authors name from the list after storing

        i_ = next((i for i, e in enumerate(details) if e.isdigit()), len(details))
        author = ' '.join(details[:i_])
       
        author = re.sub(r"(\w)(and)(\w)", r"\1 and \3", author)
        author = re.sub(r"(\w+)\s+(\w+)\s+and\s+(\w+)\s+(\w+)", r"\1 T. \2 and \3 L. \4", author)

        details = details[i_:]


        bookData = pd.concat([bookData, pd.DataFrame({
            'Title': [bookName],
            'Author': [author],
            'Year Published': [details[0]],
            'Number of Logs': [details[1]],
            'url': [soup.find('a', class_='results')['href']]
        })], ignore_index=True)
        


for i in range(10):
    processPage(f'https://openlibrary.org/trending/forever?page={i+1}')

    
bookData

,Title,Author,Year Published,Number of Logs,url
0,Atomic Habits,James Clear,41182,Time,/works/OL17930368W?edition=ia%3Aatomichabitsea...
1,It Ends With Us,Colleen Hoover,40569,Time,/works/OL17930368W?edition=ia%3Aatomichabitsea...
2,The 48 Laws of Power,Robert Greeneand Joost Elffers,34986,Time,/works/OL17930368W?edition=ia%3Aatomichabitsea...
3,The Subtle Art of Not Giving a F*ck,Mark Manson,30182,Time,/works/OL17930368W?edition=ia%3Aatomichabitsea...
4,Um casamento arranjado,Zana Kheiron,23272,Time,/works/OL17930368W?edition=ia%3Aatomichabitsea...
...,...,...,...,...,...
194,The Titan's Curse,Rick Riordan,1300,Time,/works/OL45885W?edition=ia%3Asvj0000dahl
195,Cien años de soledad,Gabriel García Márquez,1298,Time,/works/OL45885W?edition=ia%3Asvj0000dahl
196,"A child called ""it""",David J. Pelzer,1296,Time,/works/OL45885W?edition=ia%3Asvj0000dahl
197,A Wrinkle in Time,Madeleine L' Engle,1296,Time,/works/OL45885W?edition=ia%3Asvj0000dahl
